# Hierarchically Chaining the Agent Calls using a Router Agent (Let Our AI Model Guides Itself)

We will create a hierarchical workflow using a router agent. Instead of having a fixed linear workflow like in the previous lesson, by using a third agent (router) in the client-side that will decide when to call each ACP agent and navigate diverse answers from different questions.

## 8.1. Start Up both ACP Servers

First make sure the Insurer server is still running:
- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, then you don't need to do anything else.
- If the agent has stopped running (the lab environment resets after 120 min), then run the server again by typing:
  - `uv run crew_agent_server.py`


**Note**: If the platform is installed, it runs by default on port 8333. The ACP servers are configured to automatically connect to the platform. Since the platform is not installed in this environment, the ACP server will generate a warning.

In [1]:
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/1", width=800, height=600)

Also make sure the Hospital server is still running:
- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, don't need to do anything else.
- If the agent has stopped running (the lab environment resets after 120 min), run the server again by typing:
  - `uv run smolagents_server.py`

In [4]:
IFrame(f"{url}terminals/2", width=800, height=600)

## 8.2. Import ACPCallingAgent

In [1]:
# run our server
import asyncio
# nest the server
import nest_asyncio
# discover which agent is available for each question 
from acp_sdk.client import Client 
# access lite LLM Model that we always use for this ACP
from smolagents import LiteLLMModel
# the main structure for hierarchical flow. AgentCollection is a help use to format the ACP server that we're going to use them. ACPCallingAgent is our router agent (navigate the best ACP to do the action). 
from fastacp import AgentCollection, ACPCallingAgent 
# terminal formatting for output
from colorama import Fore

In [2]:
print(ACPCallingAgent.__doc__)


    This agent uses JSON-like ACP agent calls, similarly to how ToolCallingAgent uses tool calls,
    but directed at remote ACP agents instead of local tools.
    
    Args:
        acp_agents (`dict[str, Agent]`): ACP agents that this agent can call.
        model (`Callable[[list[dict[str, str]]], ChatMessage]`): Model that will generate the agent's actions.
        prompt_templates ([`Dict[str, str]`], *optional*): Prompt templates.
        planning_interval (`int`, *optional*): Interval at which the agent will run a planning step.
        **kwargs: Additional keyword arguments.
    


## 8.3. Run the Hierarchical Workflow 

In [3]:
nest_asyncio.apply()

**Note**: The `fastacp.py` file does not only contain the definition for the ACPCallingAgent, but it also includes this method: `AgentCollection.from_acp` where the client objects (`insurer` and `hospital`) discover the agents hosted on their corresponding servers by calling the method `.agents()`.

The hierarchical workflow has pretty similar to the sequential call. The difference is rather just put the prompt directly into it, we let the ACP Calling Agent discover which are the best servers to answer users question.

In [7]:
# Build the model
model = LiteLLMModel(
    model_id="openai/gpt-4"
)

# define the hospital workflow
async def run_hospital_workflow() -> None:
    async with Client(base_url="http://localhost:8001") as insurer, Client(base_url="http://localhost:8000") as hospital:
        
        # agents discovery
        agent_collection = await AgentCollection.from_acp(insurer, hospital)  # pass through all of different ACP clients
        acp_agents = {agent.name: {'agent':agent, 'client':client} for client, agent in agent_collection.agents}
        print(acp_agents) 
        
        # Define new agent (router agent), passing the agents as tools to ACPCallingAgent
        acpagent = ACPCallingAgent(acp_agents=acp_agents, model=model)
        
        # running the agent with a user query
        result = await acpagent.run("do i need rehabilitation after a shoulder reconstruction and what is the waiting period from my insurance?")
        print(Fore.YELLOW + f"Final result: {result}" + Fore.RESET)

In [8]:
asyncio.run(run_hospital_workflow())

{'policy_agent': {'agent': Agent(name='policy_agent', description='This is an agent for questions around policy coverage, it uses a RAG pattern to find answers based on policy documentation. Use it to help answer questions on coverage and waiting periods.', metadata=Metadata(annotations=None, documentation=None, license=None, programming_language=None, natural_languages=None, framework=None, capabilities=None, domains=None, tags=None, created_at=None, updated_at=None, author=None, contributors=None, links=None, dependencies=None, recommended_models=None)), 'client': <acp_sdk.client.client.Client object at 0x7f4f3f2f8550>}, 'health_agent': {'agent': Agent(name='health_agent', description='This is a CodeAgent which supports the hospital to handle health based questions for patients. Current or prospective patients can use it to find answers about their health and hospital treatments.', metadata=Metadata(annotations=None, documentation=None, license=None, programming_language=None, natura

**Optional Reading:** Here's how the hierarchical flow works using the provided file `fastacp.py`:

1. The agents hosted on each server are first discovered by their corresponding client objects and then converted to tools for the router agent (ACPCallingAgent): 
  
  <img src="hr_1.png" width="650">

2. When the router agent receives a user query, it breaks downs the query into smaller steps where each step can be executed by the specialized agent. For a given step, the router agent uses the client of the specialized agent to send the request to it:
  
  <img src="hr_2.png" width="650">

## 8.4. Resources

- [Compose Agents](https://agentcommunicationprotocol.dev/how-to/compose-agents)
- [Chained Agents as a Python file](https://github.com/nicknochnack/ACPWalkthrough/blob/main/5.%20Chained%20Agents.py)

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To access the <code>my_acp_project</code> folder:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. 

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>